In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

np.random.seed(7)

Учет сегментов может ускорить эксперименты.  
В байесовском моделировании используют иерархические модели.  
https://en.wikipedia.org/wiki/Bayesian_hierarchical_modeling  



Пример: конверсии  
-сегменты вместе  
-по отдельности  
-и. модель  

Есть K сегментов и 2 группы.  
Вначале 2 сегмента.  
Изменение может влиять на сегменты по-разному.
$$
p_A = 0.1, \quad p_B = 0.11
\\
p_{A2} = 0.8, \quad p_{B2} = 0.9
$$

Генерируются данные - $N$ попыток, $n_s$ успехов в каждой группе (4 набора $N, n_s$).  

Апостериорные распределения
$$
\begin{split}
P(n_s, N | p) & = \mbox{Binom}(n_s, N | p)
\\
P(p) & = \mbox{Beta}(p; \alpha, \beta)
\\
P(p | n_s, N) & = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

Можно объединить сегменты. Тогда 2 бета-распределения
$$
N_A = N_{A1} + N_{A2}, \quad N_B = N_{B1} + N_{B2}
\\
s_A = s_{A1} + s_{A2}, \quad s_B = s_{B1} + s_{B2}
$$

Можно считать по-отдельности. Тогда 4 бета-распределения
$$
N_{A1}, s_{A1}, \quad N_{B1}, s_{B1}
\\
N_{A2}, s_{A2}, \quad  N_{B2}, s_{B2}
$$

Можно собрать иерархическую модель.  
Считать, что $p$ разных сегментов связаны.  
Как записать априорное, правдоподобие апостериорное?  
Правдоподобие не меняется - биномиальное с p от сегмента. 
Априорное поменяется - произведение бета от p на что-то для p.

$$
\begin{split}
P(n_s, N | p_k) & = \mbox{Binom}(n_s, N | p_k) = C_{N}^{n_s} p_k^{n_s} (1-p_k)^{N-n_s}
\\
p_k \sim ? 
\\
P(p_k) & = \mbox{Beta}(p_k; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} p_k^{\alpha-1} (1 - p_k)^{\beta-1}
\\
P(p_k | n_s, N) & = \mbox{Beta}(p_k; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

$$
\mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\qquad
\mbox{Beta}(x; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\qquad
\mbox{Norm}(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
$$

In [ ]:
pa = 0.1
pb = pa * 1.05

pa2 = pa * 0.8
pb2 = pb * 0.8

N = 10000

sa = stats.binom.rvs(p=pa, n=N)
sb = stats.binom.rvs(p=pb, n=N)
sa2 = stats.binom.rvs(p=pa2, n=N)
sb2 = stats.binom.rvs(p=pb2, n=N)

print(sa, sb, sa2, sb2)

sc_a = sa + sa2
sc_b = sb + sb2
Nc_a = 2 * N
Nc_b = 2 * N

p_dist_c_a = stats.beta(a=sc_a+1, b=Nc_a-sc_a+1)
p_dist_c_b = stats.beta(a=sc_b+1, b=Nc_b-sc_b+1)

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist_c_a.pdf(x), line_color='black', name='А'))
fig.add_trace(go.Scatter(x=x, y=p_dist_c_b.pdf(x), line_color='black', opacity=0.3, name='Б'))
fig.update_layout(title='Апостериорные распределения, объединенные сегменты',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()


npost = 100000
samp_с_a = p_dist_c_a.rvs(size=npost)
samp_с_b = p_dist_c_b.rvs(size=npost)
samp_diff_p = np.sum(samp_с_b - samp_с_a > 0) / npost
print(f"Combined segm, P(p_b > p_a) post samples: {samp_diff_p}")

p_dist_a1 = stats.beta(a=sa+1, b=N-sa+1)
p_dist_b1 = stats.beta(a=sb+1, b=N-sb+1)
p_dist_a2 = stats.beta(a=sa2+1, b=N-sa2+1)
p_dist_b2 = stats.beta(a=sb2+1, b=N-sb2+1)

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist_a1.pdf(x), line_color='black', name='А1'))
fig.add_trace(go.Scatter(x=x, y=p_dist_b1.pdf(x), line_color='black', opacity=0.3, name='Б1'))
fig.add_trace(go.Scatter(x=x, y=p_dist_a2.pdf(x), line_color='black', name='А2'))
fig.add_trace(go.Scatter(x=x, y=p_dist_b2.pdf(x), line_color='black', opacity=0.3, name='Б2'))
fig.update_layout(title='Апостериорные распределения, отдельные сегменты',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

npost = 100000
samp_a1 = p_dist_a1.rvs(size=npost)
samp_b1 = p_dist_b1.rvs(size=npost)
samp_diff_p1 = np.sum(samp_b1 - samp_a1 > 0) / npost
print(f"Seg1, P(p_b > p_a) post samples: {samp_diff_p1}")
samp_a2 = p_dist_a2.rvs(size=npost)
samp_b2 = p_dist_b2.rvs(size=npost)
samp_diff_p2 = np.sum(samp_b2 - samp_a2 > 0) / npost
print(f"Seg2, P(p_b > p_a) post samples: {samp_diff_p2}")

Что дает сегментация? Когда она оправдана?  
Что с парадоксом Симпсона?

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | n_s, N) 
\\
& \propto \mbox{Binom}(n_s, N | p) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

Конверсии

Реализация этих закономерностей зависит от конкретных метрик. Далее будут рассматриваться конверсии. Конверсии можно моделировать последовательностью случайных величин с двумя возможными исходами - успехом и неудачей. Для оценки вероятности успеха $p$ в серии $N$ испытаний с $n_s$ успехами правдоподобие удобно задавать биномиальным распределением $P(\mathcal{D} | \mathcal{H}) = \mbox{Binom}(n_s, N | p)$, априорное распределение - бета-распределением $P(\mathcal{H}) = \mbox{Beta}(p; \alpha, \beta)$ с параметрами $\alpha, \beta$. Тогда апостериорная вероятность также будет бета-распределением с обновленными параметрами $P(\mathcal{H} | \mathcal{D}) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)$. При достаточно большом количестве данных $N \gg n_s \gg \alpha, \beta$ бета-распределение близко нормальному $\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) \approx \mbox{Norm}(x; \mu, \sigma^2), \, \mu = n_s / N, \, \sigma^2 = \mu (1 - \mu) / N$.

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p)
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta)
$$

$$
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
$$


$$
N \gg n_s \gg \alpha, \beta: \quad
\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) 
\approx \mbox{Norm}(x; \mu, \sigma^2),
\quad
\mu = n_s / N, 
\,
\sigma^2 = \mu (1 - \mu) / N
$$


$$
\mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\qquad
\mbox{Beta}(x; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\qquad
\mbox{Norm}(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
$$